# Hello Bluesky: 'Counting' a detector

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

### Set up plotting

In [0]:
%matplotlib notebook

# Make plots live-update while scans run.
from bluesky.utils import install_nb_kicker
install_nb_kicker()

### Instantiate objects that communicate with hardware

If we had real hardware communicating over EPICS, we would use the ``ophyd`` package integerate the hardware with bluesky. Examples:

```python
import ophyd

x_motor = ophyd.EpicsMotor('PV:...', name='x_motor')
sclr = ophyd.EpicsScaler('PV:...', name='sclr')
area_det = ophyd.AreaDetector('PV:...', name='area_det')
```

These will be covered in more detail in a later tutorial. For this tutorial, we'll use simulated hardware that it included in bluesky itself.

In [0]:
# Import a simulated detector.
from bluesky.examples import noisy_det

We can see that the `det` object has one field, named `'det'`

In [0]:
noisy_det.describe()

### Instanitate a RunEngine and a LiveTable

In [0]:
from bluesky import RunEngine
from bluesky.callbacks import LiveTable, LivePlot

RE = RunEngine({})
table = LiveTable(['det'])

### Import a 'plan'

In [0]:
from bluesky.plans import count

Put the cursor after ``import`` and hit the TAB key to see a list of plans. Or, better yet, see [this page of the bluesky documentation](https://nsls-ii.github.io/bluesky/plans.html).

In [0]:
from bluesky.plans import 

## Data Acquisition

### Executing a `count` plan with various parameters

In [0]:
RE(count([noisy_det]), table)

The return value is a list of the run IDs that uniquely identify this data set. The "scan num" is easier to remember but is not good for long-term reference because it may not be unique.

In [0]:
count?

In [0]:
# five consecutive readings
RE(count([noisy_det], num=5), table)

In [0]:
# add a plot (not a very exciting plot...)
RE(count([noisy_det], num=5), [table, LivePlot('det')])

In [0]:
# five sequential readings separated by a 1-second delay
RE(count([noisy_det], num=5, delay=1), table)

In [0]:
# a variable delay
RE(count([noisy_det], num=5, delay=[1, 2, 3, 4]), table)

In [0]:
# multiple detectors
from bluesky.examples import det1, det2
bigger_table = LiveTable(['det1', 'det2'])

RE(count([det1, det2], num=3), bigger_table)

### Composing plans

In [0]:
def multi_count(detectors):
    "Three runs with 1, 2, 3 readings each."
    for i in [1, 2, 3]:
        yield from count(detectors, num=i)

In [0]:
RE(multi_count([noisy_det]), table)

Notice that `count` is invoked three times, and that each invocation generates a new scan number and run ID.

### Choosing useful inputs

If, for your application, you are always counting the same detector, it might make more sense to hard-code it the definition of the plan.

In [0]:
def multi_count_det():
    "Three runs with 1, 2, 3 readings each."
    detectors = [noisy_det]
    for i in [1, 2, 3]:
        yield from count(detectors, num=i)

## Exercises

1. Count a detector with 3 sequential readings, generating one run.
2. Count a detector 3 times with one reading each, generating a separate run (i.e., a separate 'scan num') for each reading.
3. Count a detector with exponentially increasing delays between readings.